In [70]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/jialing/Downloads/ResidentPopulationAged15YearsandOverbyReligionMaritalStatusandSexGeneralHouseholdSurvey2015 (2).csv")

In [71]:
df.columns.tolist()

['Thousands',
 'Total_Total',
 'Total_Males',
 'Total_Females',
 'Single_Total',
 'Single_Males',
 'Single_Females',
 'Married_Total',
 'Married_Males',
 'Married_Females',
 'Widowed_Total',
 'Widowed_Males',
 'Widowed_Females',
 'Divorced_Separated_Total',
 'Divorced_Separated_Males',
 'Divorced_Separated_Females']

In [72]:
df.rename(columns={'Thousands': 'Religion Type'}, inplace=True)

In [73]:
df = df.drop(df.columns[[1, 2, 3,4, 7,10,13]], axis=1)

In [74]:
df.columns.tolist()

['Religion Type',
 'Single_Males',
 'Single_Females',
 'Married_Males',
 'Married_Females',
 'Widowed_Males',
 'Widowed_Females',
 'Divorced_Separated_Males',
 'Divorced_Separated_Females']

In [75]:
df

,Religion Type,Single_Males,Single_Females,Married_Males,Married_Females,Widowed_Males,Widowed_Females,Divorced_Separated_Males,Divorced_Separated_Females
0,Total,533.2,501.3,984.7,960.9,28.9,143.2,42.7,80.9
1,No Religion,128.9,107.6,175.4,155.7,4.2,10.6,8.7,13.4
2,With Religion,404.3,393.8,809.4,805.2,24.8,132.6,34,67.5
3,Buddhism,160.6,149.4,336.6,333.2,10.3,54.3,14.9,28.0
4,Taoism1/,45.4,42.2,107.3,95.4,4.2,22.8,3.8,5.2
5,Islam,84.3,70.8,129.9,128.3,5.2,20.4,6.6,14.4
6,Hinduism,18.8,17.3,60.2,53.4,0.9,7.0,1.5,3.5
7,Sikhism,1.5,1.7,3.5,3.4,na,0.9,0.2,0.3
8,Christianity,92.4,111.2,169.3,189.4,4.1,26.8,6.9,15.9
9,Catholic,33.5,37.1,60.0,69.3,1.5,10.2,3.1,6.2


In [76]:
# Melt all marital/gender columns into two columns: 'variable' and 'Count'
df_tidy = df.melt(
    id_vars='Religion Type',
    var_name='Status_Gender',
    value_name='Count'
)
df_tidy

,Religion Type,Status_Gender,Count
0,Total,Single_Males,533.2
1,No Religion,Single_Males,128.9
2,With Religion,Single_Males,404.3
3,Buddhism,Single_Males,160.6
4,Taoism1/,Single_Males,45.4
...,...,...,...
91,Sikhism,Divorced_Separated_Females,0.3
92,Christianity,Divorced_Separated_Females,15.9
93,Catholic,Divorced_Separated_Females,6.2
94,Other Christians,Divorced_Separated_Females,9.7


In [77]:
# Fix inconsistent names
df_tidy['Status_Gender'] = df_tidy['Status_Gender'].replace({
    'Divorced_Separated_Males': 'Divorced_Males',
    'Divorced_Separated_Females': 'Divorced_Females'
})

In [78]:
# Separate the 'Status_Gender' into two columns (Marital Status, Gender)
df_tidy[['Marital Status', 'Gender']] = df_tidy['Status_Gender'].str.split('_', expand=True)


In [79]:
df_tidy.drop(columns=['Status_Gender'], inplace=True)

In [80]:
df_tidy.rename(columns={'Count': 'Value (Thousands)'}, inplace=True)

In [81]:
df_tidy['Gender'] = df_tidy['Gender'].replace({
    'Males' : "Male",
    "Females" : "Female"})


In [82]:
df_tidy['Religion Type'] = df_tidy['Religion Type'].replace({
    "Taoism1/" : "Taoism"})

In [83]:
df_tidy.drop(index=0, inplace=True)
df_tidy
df_tidy.to_csv("tidy_religion_marital_status.csv", index=False)